In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lib import sig_gen
import tensorflow as tf
from tensorflow.keras import layers

print(tf.VERSION)
print(tf.keras.__version__)

1.10.0
2.1.6-tf


In [32]:
def sig_batch_gen(batch, symb, sps):
    """
    generate a signal batch
    no distortion yet
    """
    mibt = np.random.random((batch, 2))+0.3
    sig = np.empty((batch, symb*sps, 2))
    for idx in range(batch):
        msg = np.random.randint(0, 2, symb)
        sig_cpx = sig_gen.gfsk_mod(msg, sps, mibt[idx, 0], mibt[idx, 1])
        sig[idx, :, 0] = sig_cpx.real
        sig[idx, :, 1] = sig_cpx.imag
    return sig, mibt

# sig, mibt = sig_batch_gen(batch, symb, sps)
# print(sig.shape, mibt.shape)
def train_generator():
    batch = 1024
    symb, sps = 64, 4
    while True:
        yield sig_batch_gen(batch, symb, sps)
        
# for item in train_generator():
#     print(item)
#     if item > 4:
#     break
gen = train_generator()
next(gen)[1].shape

(1024, 2)

In [22]:
mibt[idx, 1]

0.4775702545731691

In [13]:
def build_model(input_shape):
    # build model
    model = tf.keras.Sequential()
    model.add(layers.Conv1D(32, 9, activation='relu',
                                     input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(16, 7, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(8, 3, activation='relu'))
    model.add(layers.Flatten())

    model.add(layers.Dense(2))

    
    loss_function = tf.keras.losses.mean_squared_error
    metrics = ['mae', 'acc']

    model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss=loss_function,
                  metrics=metrics)
    model.summary()

    return model

model = build_model((256, 2))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 248, 32)           608       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 124, 32)           0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 118, 16)           3600      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 59, 16)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 57, 8)             392       
_________________________________________________________________
flatten_4 (Flatten)          (None, 456)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 914       
Total para

In [16]:
# sanity check
x_test = np.random.rand(5, 256, 2)
model.predict(x_test).shape

(5, 2)

In [33]:
history = model.fit_generator(
    gen,
    steps_per_epoch=100,
    epochs=10
    )

Epoch 1/10
100/100 [==============================] - 30s 299ms/step - loss: 0.1015 - mean_absolute_error: 0.2244 - acc: 0.6386
Epoch 2/10
100/100 [==============================] - 29s 291ms/step - loss: 0.0436 - mean_absolute_error: 0.1440 - acc: 0.7508
Epoch 3/10
100/100 [==============================] - 29s 291ms/step - loss: 0.0423 - mean_absolute_error: 0.1382 - acc: 0.7530
Epoch 4/10
100/100 [==============================] - 29s 291ms/step - loss: 0.0416 - mean_absolute_error: 0.1359 - acc: 0.7556
Epoch 5/10
100/100 [==============================] - 29s 291ms/step - loss: 0.0399 - mean_absolute_error: 0.1319 - acc: 0.7605
Epoch 6/10
100/100 [==============================] - 29s 291ms/step - loss: 0.0350 - mean_absolute_error: 0.1216 - acc: 0.7858
Epoch 7/10
100/100 [==============================] - 29s 293ms/step - loss: 0.0262 - mean_absolute_error: 0.1013 - acc: 0.8337
Epoch 8/10
100/100 [==============================] - 30s 301ms/step - loss: 0.0191 - mean_absolute_erro